In [44]:
import time
import random
from ortools.linear_solver import pywraplp  # Google OR Tools - Linear Solver

solver = pywraplp.Solver.CreateSolver('SCIP')  # Solver object to use "SCIP" algorithm of the linear solver 

# Problem Data
* Lim wants to find the optimal intake quantity for SKU-A, which minimizes the total cost of inventory replenishment.
* Information on SKU-A
    - There are 4 units in the warehouse in the end of today.
    - It is being sold at S$10.
    - We have its demand forecast for the next 10 days.
    - We're going to plan the intake of the next 10 days.
* Information on SKU-A's supplier
    - Minimum order quantity (MOQ): It only accepts orders with at least 10 units.
    - Fixed delivery cost: Each order costs $20 for delivery.

In [50]:
# SKU
stock_level = 4  # As of end of D+0
selling_price = 10  # SGD
number_of_days = 10  # N days, where N = 10 in this example
demand = [3, 5, 3, 5, 3, 5, 3, 5, 3, 5]  # [D+1, D+2, ..., D+10]

inventory_holding_cost = selling_price * 0.01  # Inventory holding cost per unit
lost_sales_cost = selling_price  # Lost sales cost per unit

# Supplier
moq = 10  # Minimum order quantity
delivery_cost = 1  # Fixed delivery cost

# Variables
* (Non-negative) binary decision variable: x[i] = 1 (if intake exists) or 0 (if no intake)
    - We have N days, so we have N binary decision variables, x[1], x[2], ..., x[N].
* (Non-negative) integer decision variable: intake[i] > 0 if intake exists or 0 otherwise
    - If x[i] = 1, intake[i] > 0. Otherwise, intake[i] = 0
* (Non-negative) integer decision variable: inv[i] > 0 if there are any remaining units in the end of the i-th day or 0 otherwise
* (Non-negative) integer decision variable: lost[i] > 0 if there are any unmet demand on the i-th day or 0 otherwise
    - At least one of inv[i] and lost[i] is zero.

In [51]:
x = {}  # Binary decision variable, x[i] = 1 if intake quantity in the beginning of the i-th day is positive or 0 otherwise
intake = {}  # Integer decision variable, intake[i] = intake quantity in the beginning of the i-th day
inv = {}  # Integer decision variable, inv[i] = inventory level (in quantity) in the end of the i-th day
lost = {}  # Integer decision variable, lost[i] = lost sales quantity in the end of the i-th day
for i in range(number_of_days):
    x[i] = solver.BoolVar(f'x_{i}')
    intake[i] = solver.IntVar(0, solver.infinity(), f'ord_{i}')
    inv[i] = solver.IntVar(0, solver.infinity(), f'inv_{i}')
    lost[i] = solver.IntVar(0, solver.infinity(), f'lost_{i}')

# Constraints
* Intake quantity on a day can be positive only if an intake happens on the day.
* Intake quantity must be no smaller than the minimum order quantity.
    - MOQ * x[i] ≤ intake[i] ≤ 1,000,000 * x[i] for i = 1, 2, ..., N
    - If x[i] = 1
        - MOQ ≤ intake[i] ≤ 1,000,000
        - Hence, intake[i] can be any positive integer value no smaller than MOQ
    - If x[i] = 0
        - 0 ≤ intake[i] ≤ 0
        - Hence, intake[i] = 0
* Inventory Dynamics: calculation of today's ending inventory level is depending on yesterday's ending inventory level and today's intake & demand.
    - inv[i-1] + intake[i] - d[i] + lost[i] = inv[i] for i = 1, 2, ..., N
        - inv[0] = stock_level (= 4)
    - lost[i] ≥ 0 when demand is greater than (previous day's inventory + intake today) because inv[i] ≥ 0

In [52]:
# Constraint (1): intake quantity can be positive only when there is an intake
for i in range(number_of_days):
    solver.Add(
        intake[i] <= 1000000 * x[i]
    )

# Constraint (2): minimim order quantity (MOQ)
for i in range(number_of_days):
    solver.Add(
        intake[i] >= moq * x[i]
    )

# Constraint (3): inventory dynamics
for i in range(number_of_days):
    if i == 0:
        solver.Add(
            stock_level + intake[i] - demand[i] + lost[i] == inv[i]
        )
    else:
        solver.Add(
            inv[i-1] + intake[i] - demand[i] + lost[i] == inv[i]
        )

# Objective Function
* Lim wants to minimize the total cost of inventory replenishment.
* So, the objective function needs to take the delivery cost, inventory holding cost & lost sales cost in its calculation.
    - Minimize sum{ delivery_cost * x[i] + holding_cost * inv[i] + lost_sales_cost * lost[i] | i = 1, 2, ..., N }

In [53]:
# Minimize the total cost of inventory
solver.Minimize(
    sum(
        (delivery_cost * x[i]) + (inventory_holding_cost * inv[i]) + (selling_price * lost[i])
        for i in range(number_of_days)
    )
)

# Solve
### Linear Programming Model
- Minimize
    - Minimize sum{ delivery_cost * x[i] + holding_cost * inv[i] + lost_sales_cost * lost[i] | i = 1, 2, ..., N }
- Subject to
    - (1) intake[i] ≤ 1,000,000 * x[i] for i = 1, 2, ..., N
    - (2) MOQ * x[i] ≤ intake[i] for i = 1, 2, ..., N
    - (3) inv[i-1] + intake[i] - demand[i] + lost[i] = inv[i] for i = 1, 2, ..., N
    - x[i] ∈ {0, 1} for i = 1, 2, ..., N
    - intake[i] ≥ 0 & integer for i = 1, 2, ..., N
    - inv[i] ≥ 0 & integer for i = 1, 2, ..., N
    - lost[i] ≥ 0 & integer for i = 1, 2, ..., N

In [58]:
# Solve the linear program
status = solver.Solve()

# Check the results
print('status =', status)  # 0=optimal, 1=feasible, 2=infeasible, 3=unbounded, 4=abnormal, 6=not solved
print(f'objective value = {solver.Objective().Value()}\n')

# Print figures in a table form
print(f'initial stock level = {stock_level}')
val = 'day'
for i in range(number_of_days):
    val += '\t' + str(i+1)
print(val)
val = 'intake'
for i in range(number_of_days):
    val += '\t' + str(int(intake[i].solution_value()))
print(val)
val = 'demand'
for i in range(number_of_days):
    val += '\t' + str(demand[i])
print(val)
val = 'inv'
for i in range(number_of_days):
    val += '\t' + str(int(inv[i].solution_value()))
print(val)
val = 'lost'
for i in range(number_of_days):
    val += '\t' + str(int(lost[i].solution_value()))
print(val)

# Total figures
total_intake_qty = 0
total_inventory = 0
total_lost_sales_qty = 0
for i in range(number_of_days):
    if x[i].solution_value() == 1:
        total_intake_qty += intake[i].solution_value()
        total_inventory += inv[i].solution_value()
        total_lost_sales_qty += lost[i].solution_value()
print('\ntotal intake quantity =', int(total_intake_qty))
print('total inventory quantity =', int(total_inventory))
print('total lost sales quantity =', int(total_lost_sales_qty))

status = 0
objective value = 6.800000000000001

initial stock level = 4
day	1	2	3	4	5	6	7	8	9	10
intake	0	12	0	0	11	0	0	13	0	0
demand	3	5	3	5	3	5	3	5	3	5
inv	1	8	5	0	8	3	0	8	5	0
lost	0	0	0	0	0	0	0	0	0	0

total intake quantity = 36
total inventory quantity = 24
total lost sales quantity = 0
